In [2]:
from bs4 import BeautifulSoup
import lxml #is a parser for HTMl
import webscraping
import requests
import os
import re
import csv
#from selenium import webdriver
from langdetect import detect
import pandas as pd

We shoul generalize these functions... in webscraping.py!

## **[RQ1.1] Get the list of books**

We start to scrape the three pages to extract the URLs for each of them

In [23]:
UrlsFiles = open("urlpages.txt", "w")

for i in range(1, 101):
    url = "https://www.goodreads.com/list/show/1.Best_Books_Ever?page="+str(i)
    
    page = requests.get(url)
    soup = BeautifulSoup(page.content, features='lxml')
    for a in soup.find_all('a', class_="bookTitle"):
        UrlsFiles.write(a.get('href')+'\n')

UrlsFiles.close()

## **RQ[1.2] Crawl books**

In [24]:
headpart = "https://www.goodreads.com"

In [25]:
direct = "htmlpages" #Directory  
parent_dir = "./" #Parent Directory path 
pathAncestor = os.path.join(parent_dir, direct) #Path
os.mkdir(pathAncestor) #create the folder in the path

In [26]:
for i in range(1,101):
    os.makedirs(os.path.join(pathAncestor, 'page ' + str(i)))

In [34]:
UrlsFiles = open("urlpages.txt", "r")

counter_pages = 0
counter_html = 0
for x in UrlsFiles:
    if counter_html % 100 == 0:
        counter_pages = counter_pages+1
        
    counter_html = counter_html + 1
    
    subdirectory = pathAncestor + "/page " + str(counter_pages)
    article_name = "/article_"+str(counter_html)+".html"
    
    complete_path = subdirectory + article_name
    with open(complete_path, "wb") as ip_file:
        link = headpart + x
        try:
            page = requests.get(link)
        except:
            with open("failureRequest.txt", "wb") as err_file:
                err_file.write(link)
                err_file.close()
        
        soup = BeautifulSoup(page.text, features='lxml')
        
        ip_file.write(soup.encode('utf-8'))
        ip_file.close()

UrlsFiles.close()

## **RQ[1.3] Parse downloaded pages**

In [20]:
def scrap_book(tsv_writer, article):
    global bookTitle, bookSeries, bookAuthors, ratingValue, ratingCount, reviewCount, Plot, NumberofPages, Published, Characters, Setting, Url;
    
    with open(article, 'r', encoding="utf-8") as out_file:
        contents = out_file.read()
        soup = BeautifulSoup(contents, features="lxml") #parse the text

        #extract rating and review count
        try:
            ratings = soup.find_all('a', href="#other_reviews") #search the ratings in its
            rating_count = -1
            rating = -1
            for raiting in ratings:
                if raiting.find_all('meta', itemprop="ratingCount"):
                    ratingCount = raiting.text.replace('\n', '').strip().split(' ')[0].replace(',', '')
                elif raiting.find_all('meta', itemprop="reviewCount"):
                    reviewCount = raiting.text.replace('\n', '').strip().split(' ')[0].replace(',', '')
        except:
            ratingCount = " "
            reviewCount = " "
            
        #extract the book title
        try:
            bookTitle = soup.find_all('h1')[0].contents[0].replace('\n', '').strip()
        except:
            bookTitle = " "

        #extract the book authors
        try:
            bookAuthors = soup.find_all('span', itemprop='name')[0].contents[0]
        except:
            bookAuthors = " "

        #extract the book authors, we shoul FIX it.
        try:
            Plot = soup.find_all('div', id="description")[0].contents[3].text
            if detect(Plot) != "en":
                Plot = " "
        except:
            try:
                Plot = soup.find_all('div', id="description")[0].contents[1].text
                if detect(Plot) != "en":
                    Plot = " "
            except:
                Plot = " "
                

        #extract the date
        try:
            date = soup.find_all('div', id="details")[0].contents[3].text.replace('\n', '').strip().split()
            Published = date[1]+" "+date[2]+" "+date[3]
        except:
            Published = " "

        #Rating Value
        try:
            ratingValue = soup.find('span', itemprop="ratingValue").text.strip()
        except:
            ratingValue = " "

        #Number of pages
        try:
            NumberofPages = soup.find('span', itemprop="numberOfPages").text.split()[0]
        except:
            NumberofPages = " "

        #Title series
        try:
            bookSeries = soup.find_all('a', href= re.compile(r'/series/*'))[0].contents[0].strip()
        except:
            bookSeries = " "
            
        #Places
        try:
            Setting = []
            for places in soup.find_all('a', href= re.compile(r'/places/*')):
                Setting.append(places.text)
            Setting = ", ".join(Setting) if len(Setting)>=1 else " "
        except:
            Setting = " "

        #list of characters
        try:
            Characters = []
            for character in soup.find_all('a', href= re.compile(r'/characters/*') ):
                Characters.append(character.text)
            Characters = ", ".join(Characters) if len(Characters)>=1 else " "
        except:
            Characters = " "

        #extract the Url
        try:
            Url = soup.find_all('link')[0]["href"]
        except:
            Url = " "

        tsv_writer.writerow([bookTitle, bookSeries, bookAuthors, ratingValue, ratingCount, reviewCount, Plot, NumberofPages, Published, Characters, Setting, Url])

In [22]:
path = "./html_pages"

filenames = os.listdir(path)
for i in range(1, 101):
    filenames = os.listdir(path + '/' + str(i))

    for file in filenames:
        with open(path + '/' + str(i) + './article_'+str(file.split("_")[1].replace(".html", ""))+'.tsv', 'w', encoding="utf-8", newline='') as out_file:
            tsv_writer = csv.writer(out_file, delimiter='\t')
            tsv_writer.writerow(['bookTitle', 'bookSeries', 'bookAuthors', 'ratingValue', 
                            'ratingCount', 'reviewCount', 'Plot', 'NumberofPages', 'Published',
                            'Characters', 'Setting', 'Url'])
            scrap_book(tsv_writer, path + '/' + str(i) + "/" + file)

### creating dataset

## RQ[2] Search Engine

In [3]:
df1 = pd.read_csv('NEWoutput1.tsv',sep="\t")
#df2 = pd.read_csv('NEWoutput2.tsv', sep="\t")
#df3 = pd.read_csv('NEWoutput3.tsv', sep="\t")

#complete_dataset = pd.concat([df1, df2, df3])
#complete_dataset = complete_dataset.reset_index(drop=True) # reset index

#complete_dataset = complete_dataset.drop_duplicates(subset=['bookTitle'])
#complete_dataset = complete_dataset.dropna(subset=['Plot'])
#complete_dataset = complete_dataset.reset_index(drop=True)
#complete_dataset

## **Cleaning data**

In [4]:
#DEF Cleaning
import nltk

from nltk.corpus import stopwords
from nltk.stem.snowball import PorterStemmer
from nltk.tokenize import RegexpTokenizer
#from nltk.stem import WordNetLemmatizer

In [5]:
def cleaningDataset(df):
    for i, row in df.iterrows():
        tokenizer = RegexpTokenizer("[\w']+")  # import the tokenizer punctuation
        
        df.at[i, 'Plot'] = tokenizer.tokenize(df.at[i, 'Plot'].lower()) # remove the punctuation
        df.at[i, 'Plot'] = [w for w in df.at[i, 'Plot'] if not w in set(stopwords.words('english'))]  # words in english to avoid few data for the cleaning data
        df.at[i, 'Plot'] = [PorterStemmer().stem(word) for word in df.at[i, 'Plot']] # contesto
    return df

In [6]:
def createVocabulary(df):
    vocabulary = {}
    for i, row in df.iterrows():
        for word in df.at[i, 'Plot']:
            if word in vocabulary.keys():
                if "document_"+str(i) not in vocabulary[word]:
                    vocabulary[word].append("document_"+str(i))
            else:
                vocabulary[word] = ["document_"+str(i)]
    return vocabulary

In [79]:
#create and clean the dataset
df = df1[0:50].copy()#complete_dataset.copy()
df = cleaningDataset(df)

vocabulary = createVocabulary(df)    

In [165]:
#create file .csv
import csv

with open('./vocabulary.tsv', 'w', encoding="utf-8", newline='') as out_file:
    tsv_writer = csv.writer(out_file, delimiter='\t')
    tsv_writer.writerow(["Word", "Term_id", "Document_List"])
    
    list_of_words = vocabulary.keys()
    for id, word in enumerate(list_of_words, 1):
        term_id_i = "term_id_"+str(id)
        tsv_writer.writerow([word, term_id_i, vocabulary[word]]) 

## RQ[2.1.2] Execute the query

In [166]:
def cleanQuery(row):
    row = row.split(" ") # split the string query
    for element in row:
        tokenizer = RegexpTokenizer("[\w']+")  # import the tokenizer punctuation
        element = tokenizer.tokenize(element.lower()) # remove the punctuation
    
    row = [w for w in row if not w in set(stopwords.words('english'))] # words in english to avoid few data for the cleaning data
    
    #row = [lemmatizer.lemmatize(word) for word in row]
    row = [PorterStemmer().stem(word) for word in row]
    
    return row

In [4]:
query = input("Insert your query string: ")

Insert your query string: Survival games


In [13]:
cleanQString = cleanQuery(query)
cleanQString

['surviv', 'game']

open the dataset vocabulary

In [214]:
vocabulary = pd.read_csv('vocabulary.tsv',sep="\t")
vocabulary = vocabulary.drop_duplicates(subset=['Word'])
vocabulary = vocabulary.reset_index(drop=True)

In [215]:
vocabulary

,Word,Term_id,Document_List
0,could,term_id_1,"['document_0', 'document_26', 'document_38']"
1,surviv,term_id_2,"['document_0', 'document_26', 'document_40', '..."
2,wild,term_id_3,"['document_0', 'document_8', 'document_19', 'd..."
3,everi,term_id_4,"['document_0', 'document_3', 'document_5', 'do..."
4,one,term_id_5,"['document_0', 'document_9', 'document_14', 'd..."
...,...,...,...
1988,manheim,term_id_1989,['document_49']
1989,1996,term_id_1990,['document_49']
1990,access,term_id_1991,['document_49']
1991,poem,term_id_1992,['document_49']


## create the first inverted_list

In [216]:
def make_it_string(word):
    for sym in ["[", "]", "'"]:
        word = word.replace(sym, "")
    word = word.split(", ")
    
    return word

In [217]:
inv_lst = {}
for i, row in vocabulary.iterrows():
    term = vocabulary.at[i, "Term_id"]
    inv_lst[term] = make_it_string(vocabulary.at[i, "Document_List"])

In [218]:
inv_lst

{'term_id_1': ['document_0', 'document_26', 'document_38'],
 'term_id_2': ['document_0', 'document_26', 'document_40', 'document_49'],
 'term_id_3': ['document_0',
  'document_8',
  'document_19',
  'document_42',
  'document_47'],
 'term_id_4': ['document_0',
  'document_3',
  'document_5',
  'document_7',
  'document_15',
  'document_24',
  'document_36',
  'document_39',
  'document_48'],
 'term_id_5': ['document_0',
  'document_9',
  'document_14',
  'document_15',
  'document_18',
  'document_19',
  'document_21',
  'document_25',
  'document_28',
  'document_30',
  'document_31',
  'document_34',
  'document_36',
  'document_37',
  'document_38',
  'document_39',
  'document_41',
  'document_42',
  'document_44'],
 'term_id_6': ['document_0',
  'document_4',
  'document_15',
  'document_31',
  'document_33',
  'document_34',
  'document_36',
  'document_47'],
 'term_id_7': ['document_0', 'document_27'],
 'term_id_8': ['document_0',
  'document_14',
  'document_15',
  'document_19

save as csv file

In [219]:
with open('inv_lst.csv', 'w') as f:
    for key in inv_lst.keys():
        f.write("%s:%s\n"%(key,inv_lst[key]))

In [8]:
#inv_lst.set_index('Word', inplace=True)

In [9]:
#inv_lst.head()

,Term_id,Document_List
Word,,
could,term_id_1,"['document_1', 'document_27', 'document_39', '..."
surviv,term_id_2,"['document_1', 'document_27', 'document_41', '..."
wild,term_id_3,"['document_1', 'document_9', 'document_20', 'd..."
everi,term_id_4,"['document_1', 'document_4', 'document_6', 'do..."
one,term_id_5,"['document_1', 'document_10', 'document_15', '..."


# Search Engine, 1st version

In [227]:
# open the inv_lst
reader = csv.reader(open('inv_lst.csv', 'r'), delimiter=":")
inv_lst = {}
for row in reader:
    k, v = row
    inv_lst[k] = v

In [229]:
for key in inv_lst:
    inv_lst[key] = make_it_string(inv_lst[key])

In [250]:
# find each term_id
term_id = []
for token in cleanQString:
    term = vocabulary.loc[vocabulary["Word"] == token, "Term_id"].values[0]
    term_id.append(term)

In [276]:
intersection_list = []
for term in term_id:
    if not intersection_list:
        intersection_list = inv_lst[term]
    else:
        intersection_list = set(intersection_list).intersection(set(inv_lst[term]))

['document_0', 'document_26', 'document_38']


In [279]:
new_df = pd.DataFrame(columns=['bookTitle', 'Plot', 'Url'])

for row in intersection_list:
    i = int(row.split("_")[1])
    
    #append row to the dataframe
    new_row = {'bookTitle': df.loc[i, "bookTitle"], 'Plot': df1.loc[i, "Plot"], 'Url': df.loc[i, "Url"]}
    new_df = new_df.append(new_row, ignore_index=True)

In [281]:
new_df

,bookTitle,Plot,Url
0,The Hunger Games,"Could you survive on your own in the wild, wit...",https://www.goodreads.com/book/show/2767052-th...


## RQ[2.2] Conjunctive query & Ranking score

In [87]:
#create the second inverted list
inv_lst = pd.read_csv('inverted_list.tsv',sep="\t")
inv_lst.head()

,Word,Term_id,Document_List
0,could,term_id_1,"['document_0', 'document_26', 'document_38']"
1,surviv,term_id_2,"['document_0', 'document_26', 'document_40', '..."
2,wild,term_id_3,"['document_0', 'document_8', 'document_19', 'd..."
3,everi,term_id_4,"['document_0', 'document_3', 'document_5', 'do..."
4,one,term_id_5,"['document_0', 'document_9', 'document_14', 'd..."


In [42]:
#create and clean the dataset
#df = df1[1:50].copy()#complete_dataset.copy()
#df = cleaningDataset(df)

In [88]:
inv_lst['Document_PairList'] = ''
inv_lst

,Word,Term_id,Document_List,Document_PairList
0,could,term_id_1,"['document_0', 'document_26', 'document_38']",
1,surviv,term_id_2,"['document_0', 'document_26', 'document_40', '...",
2,wild,term_id_3,"['document_0', 'document_8', 'document_19', 'd...",
3,everi,term_id_4,"['document_0', 'document_3', 'document_5', 'do...",
4,one,term_id_5,"['document_0', 'document_9', 'document_14', 'd...",
...,...,...,...,...
1988,manheim,term_id_1989,['document_49'],
1989,1996,term_id_1990,['document_49'],
1990,access,term_id_1991,['document_49'],
1991,poem,term_id_1992,['document_49'],


In [89]:
import math

for i, row in inv_lst.iterrows():
    lst_doc = make_it_string(inv_lst.at[i, 'Document_List'])

    result = []
    for doc in lst_doc:
        number_doc = int(doc.split("_")[1])

        interested_row = df.at[number_doc, "Plot"]
        
        interested_word = inv_lst.at[i, "Word"] #i-th word
        
        tf = interested_row.count(interested_word) / len(interested_row)
        
        idf = math.log(len(df)/len(lst_doc))

        tf_idf = round(tf * idf, 3)
       
        result.append((doc, tf_idf))
        
    inv_lst.at[i, "Document_PairList"] = result

del inv_lst["Document_List"]

In [90]:
inv_lst.set_index('Word', inplace=True)
inv_lst

,Term_id,Document_PairList
Word,,
could,term_id_1,"[(document_0, 0.033), (document_26, 0.023), (d..."
surviv,term_id_2,"[(document_0, 0.088), (document_26, 0.021), (d..."
wild,term_id_3,"[(document_0, 0.027), (document_8, 0.123), (do..."
everi,term_id_4,"[(document_0, 0.02), (document_3, 0.082), (doc..."
one,term_id_5,"[(document_0, 0.034), (document_9, 0.021), (do..."
...,...,...
manheim,term_id_1989,"[(document_49, 0.027)]"
1996,term_id_1990,"[(document_49, 0.027)]"
access,term_id_1991,"[(document_49, 0.027)]"


In [91]:
with open('./inverted_list_2.tsv', 'w', encoding="utf-8", newline='') as out_file:
    tsv_writer = csv.writer(out_file, delimiter='\t')
    tsv_writer.writerow(["Word", "Term_id", "(Document_i, tfIdf)"])
    
    list_of_words = vocabulary.keys()
    for id, word in enumerate(list_of_words, 1):
        term_id_i = "term_id_"+str(id)
        tsv_writer.writerow([word, term_id_i, inv_lst.at[word, "Document_PairList"]]) 

## RQ[2.2.2] SEARCH ENGINE 2

In [93]:
query = input("Insert your query string: ")

Insert your query string: could wild


In [94]:
cleanQString = cleanQuery(query)
cleanQString

['could', 'wild']

In [95]:
inv_lst2 = pd.read_csv('inverted_list_2.tsv',sep="\t")
inv_lst2.head()

,Word,Term_id,"(Document_i, tfIdf)"
0,could,term_id_1,"[('document_0', 0.033), ('document_26', 0.023)..."
1,surviv,term_id_2,"[('document_0', 0.088), ('document_26', 0.021)..."
2,wild,term_id_3,"[('document_0', 0.027), ('document_8', 0.123),..."
3,everi,term_id_4,"[('document_0', 0.02), ('document_3', 0.082), ..."
4,one,term_id_5,"[('document_0', 0.034), ('document_9', 0.021),..."


In [96]:
inv_lst2.set_index('Word', inplace=True)
inv_lst2

,Term_id,"(Document_i, tfIdf)"
Word,,
could,term_id_1,"[('document_0', 0.033), ('document_26', 0.023)..."
surviv,term_id_2,"[('document_0', 0.088), ('document_26', 0.021)..."
wild,term_id_3,"[('document_0', 0.027), ('document_8', 0.123),..."
everi,term_id_4,"[('document_0', 0.02), ('document_3', 0.082), ..."
one,term_id_5,"[('document_0', 0.034), ('document_9', 0.021),..."
...,...,...
manheim,term_id_1989,"[('document_49', 0.027)]"
1996,term_id_1990,"[('document_49', 0.027)]"
access,term_id_1991,"[('document_49', 0.027)]"


In [97]:
def make_it_string(word):
    for sym in ["[", "]", "'", "(", ")"]:
        word = word.replace(sym, "")
    word = word.split(", ")
    
    return list(word)

In [98]:
documents = []
for i, row in df.iterrows():
    tokens = {}
    for token in df.at[i, "Plot"]:
        pairlists = make_it_string(inv_lst2.at[token, "(Document_i, tfIdf)"])
        
        index_tfidf = pairlists.index("document_"+str(i)) + 1
        
        tf_idf = float(pairlists[index_tfidf])
        
        tokens[token] = tf_idf
        
    documents.append(tokens)    

In [143]:
import math

top_k_documents = [] #similarity
for i, row in df.iterrows():
    card_d_i = 1 / math.sqrt( sum(documents[i].values()) )
    card_q = 1 / math.sqrt(len(cleanQString))
    
    somma = 0
    for token in cleanQString:
        try:
            somma += documents[i][token]
        except:
            somma += 0
            
    cosine_similarity = card_q * card_d_i * somma
        
    top_k_documents.append([round(cosine_similarity, 2), "document_"+str(i)])

In [161]:
import heapq 
# using heapify to convert list into heap 
heapq.heapify(top_k_documents) 
show_top_k_documents = (heapq.nlargest(5, top_k_documents)) 

In [163]:
new_df = pd.DataFrame(columns=['bookTitle', 'Plot', 'Url', 'Similarity'])

for row in show_top_k_documents:
    i = int(row[1].split("_")[1])
    
    #append row to the dataframe
    new_row = {'bookTitle': df.loc[i, "bookTitle"], 'Plot': df1.loc[i, "Plot"], 'Url': df.loc[i, "Url"], 'Similarity': row[0]}
    new_df = new_df.append(new_row, ignore_index=True)

In [164]:
new_df

,bookTitle,Plot,Url,Similarity
0,Where the Wild Things Are,"Max, a wild and naughty boy, is sent to bed wi...",https://www.goodreads.com/book/show/19543.Wher...,0.11
1,The Picture of Dorian Gray,"Written in his distinctively dazzling manner, ...",https://www.goodreads.com/book/show/5297.The_P...,0.05
2,A Wrinkle in Time,It was a dark and stormy night.Out of this wil...,https://www.goodreads.com/book/show/33574273-a...,0.03
3,The Hunger Games,"Could you survive on your own in the wild, wit...",https://www.goodreads.com/book/show/2767052-th...,0.03
4,The Lightning Thief,Alternate cover for this ISBN can be found her...,https://www.goodreads.com/book/show/28187.The_...,0.01
